# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lompoc,34.6391,-120.4579,18.94,67,0,4.47,US,1699923258
1,1,novotroitsk,51.2030,58.3266,4.39,69,100,5.17,RU,1699923480
2,2,grytviken,-54.2811,-36.5092,2.29,98,100,3.44,GS,1699923481
3,3,alice springs,-23.7000,133.8833,32.75,15,4,3.60,AU,1699923446
4,4,atafu village,-8.5421,-172.5159,28.29,81,100,5.58,TK,1699923481


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City"
)

map_plot_1

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] > 25)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] > 50)]
city_data_df = city_data_df.loc[(city_data_df)["Wind Speed"] < 1]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,bengkulu,-3.8004,102.2655,25.79,87,92,0.80,ID,1699923426
39,39,antsohihy,-14.8796,47.9875,25.21,83,91,0.52,MG,1699923485
42,42,puerto maldonado,-12.6000,-69.1833,28.76,61,92,0.69,PE,1699923485
105,105,itaituba,-4.2761,-55.9836,30.09,66,100,0.00,BR,1699923494
107,107,eirunepe,-6.6603,-69.8736,25.54,86,88,0.16,BR,1699923494
152,152,karaton,-6.3424,106.1045,27.66,77,99,0.31,ID,1699923501
191,191,galesong,-5.3166,119.3661,30.01,65,99,0.75,ID,1699923506
227,227,golfito,8.6390,-83.1660,25.05,95,100,0.05,CR,1699923511
259,259,tongouson,5.0214,120.1328,28.42,70,55,0.01,PH,1699923514
304,304,ceeldheer,3.8488,47.1806,26.58,83,77,0.79,SO,1699923520


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[{"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity}]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\adamg\AppData\Local\Temp\ipykernel_33260\2468048700.py:8: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  hotel_df = city_data_df[{"City": city,
C:\Users\adamg\AppData\Local\Temp\ipykernel_33260\2468048700.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
7,bengkulu,ID,-3.8004,102.2655,87,
39,antsohihy,MG,-14.8796,47.9875,83,
42,puerto maldonado,PE,-12.6000,-69.1833,61,
105,itaituba,BR,-4.2761,-55.9836,66,
107,eirunepe,BR,-6.6603,-69.8736,86,
152,karaton,ID,-6.3424,106.1045,77,
191,galesong,ID,-5.3166,119.3661,65,
227,golfito,CR,8.6390,-83.1660,95,
259,tongouson,PH,5.0214,120.1328,70,
304,ceeldheer,SO,3.8488,47.1806,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bengkulu - nearest hotel: Dena Hotel
antsohihy - nearest hotel: Escale
puerto maldonado - nearest hotel: Puerto Palmeras
itaituba - nearest hotel: TG Hotel
eirunepe - nearest hotel: No hotel found
karaton - nearest hotel: Orange Syariah
galesong - nearest hotel: No hotel found
golfito - nearest hotel: Bahia Marina Golfito Hotel
tongouson - nearest hotel: No hotel found
ceeldheer - nearest hotel: No hotel found
muli - nearest hotel: Mulaku Atoll
kavieng - nearest hotel: Kavieng Hotel
bouna - nearest hotel: Hôtel Éléphant  Bouna
puerto el triunfo - nearest hotel: Motel Jardín
ixtapa - nearest hotel: Ma. Cristina
puerto quijarro - nearest hotel: No hotel found
masuguru - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
7,bengkulu,ID,-3.8004,102.2655,87,Dena Hotel
39,antsohihy,MG,-14.8796,47.9875,83,Escale
42,puerto maldonado,PE,-12.6000,-69.1833,61,Puerto Palmeras
105,itaituba,BR,-4.2761,-55.9836,66,TG Hotel
107,eirunepe,BR,-6.6603,-69.8736,86,No hotel found
152,karaton,ID,-6.3424,106.1045,77,Orange Syariah
191,galesong,ID,-5.3166,119.3661,65,No hotel found
227,golfito,CR,8.6390,-83.1660,95,Bahia Marina Golfito Hotel
259,tongouson,PH,5.0214,120.1328,70,No hotel found
304,ceeldheer,SO,3.8488,47.1806,83,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)